In [ ]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
from amftrack.pipeline.development.high_mag_videos.plot_data import (
    read_video_data,
    plot_summary
)
import os
import pandas as pd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from pathlib import Path
from amftrack.pipeline.launching.run_super import run_parallel_flows
import re
from amftrack.util.dbx import upload_folders, upload, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, get_dropbox_folders, get_dropbox_video_folders
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300




In [ ]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/videos/MYRISTATE/DATA/2_weeks/"
upload_targ = "/DATA/MYRISTATE/DATA/2_weeks/"

In [ ]:
test_name = glob(directory_targ + "*")[0]
imgs_address = test_name
print(test_name)

In [ ]:
plate_list = glob(directory_targ + "*_Plate*/*/")
datadict = {"tot_path_drop": plate_list}
dataframe = pd.DataFrame(data=datadict)

# print(dataframe['tot_path_drop'][0])

dataframe["video"] = [
    address[:-1].split(os.sep)[-1] for address in dataframe["tot_path_drop"]
]
dataframe["Plate number"] = [
    re.split("_|/", address.split("_")[-1])[0][5:]
    for address in dataframe["tot_path_drop"]
]
dataframe["video_int"] = [
    int(re.split("_|/", address[:-1])[-1]) for address in dataframe["tot_path_drop"]
]
dataframe["Date Imaged"] = [
    address.split("/")[-3][:8] for address in dataframe["tot_path_drop"]
]
dataframe["folder"] = [
    f"{row['Date Imaged']}_plate{row['Plate number']}_{row['video']}"
    for index, row in dataframe.iterrows()
]

print(dataframe["tot_path_drop"][0])
dataframe.sort_values(by="video")

In [ ]:
data_table = []
for row in dataframe.iloc:
    excel_file = glob(
        str(Path(row["tot_path_drop"]).parent)
        + f'/*{row["Date Imaged"]}*{row["Plate number"]}.xl*'
    )
    if len(excel_file) > 0:
        data_table.append(excel_file[0])
    else:
        csv_file = glob(
            str(Path(row["tot_path_drop"]).parent)
            + f'/{row["Date Imaged"]}*{row["Plate number"]}.csv'
        )
        if len(csv_file) > 0:
            data_table.append(csv_file[0])
excel_addresses = pd.Series(data_table).unique()
excel_addresses

In [ ]:
merge_frame = read_video_data(excel_addresses, dataframe)
merge_frame.tot_path[0]

In [ ]:
download_frame = merge_frame.rename(columns={"tot_path": "folder"})
download_frame = download_frame.sort_values("unique_id")
download_frame = download_frame.reset_index(drop=True)
# dataframe_filtered = dataframe_filtered[dataframe_filtered['date_imaged'] == "20221026"]
# dataframe_filtered['plate_nr'].unique()
dataframe_filtered

In [ ]:
# FINAL CHECK FOR SOURCE FOLDER AND UPLOAD FOLDER

# Please make sure that the upload folder is correct,
# as the program WILL overwrite that which is already there.
print(directory_targ)
print(upload_targ)

In [ ]:
nr_parallel = np.min([len(dataframe.index), 16])

run_parallel_flows(
    "flux_extract.py",
    [directory_targ, 15, 0.95, 0.001, 60, upload_targ],
    dataframe_filtered,
    nr_parallel,
    "1:00:00",
    "flux_extract",
)

Upload the proper excel files to the analysis folder.

In [ ]:
parent_folder = []
data_table = []
magnification = []
fps = []
mode = []
binning = []


for row in dataframe.iloc:
    parent_folder.append(str(Path(row["tot_path_drop"]).parent))
    excel_file = glob(
        str(Path(row["tot_path_drop"]).parent)
        + f'/*{row["Date Imaged"]}*{row["Plate number"]}.xl*'
    )
    if len(excel_file) > 0:
        data_table.append(excel_file[0])
        excel_table = pd.read_excel(excel_file[0])
        magnification.append(
            excel_table[
                excel_table["Unnamed: 0"].str.contains(
                    row["video_name"], case=False, na=False
                )
            ]["Magnification"].iloc[0]
        )
        fps.append(
            excel_table.loc[
                excel_table["Unnamed: 0"].str.contains(
                    row["video_name"], case=False, na=False
                )
            ]["FPS"].iloc[0]
        )
        mode.append(
            excel_table.loc[
                excel_table["Unnamed: 0"].str.contains(
                    row["video_name"], case=False, na=False
                )
            ]["Bright-field (BF)\nor\nFluorescence (F)"].iloc[0]
        )
        if "Binned (Y/N)" in excel_table:
            binning.append(
                excel_table.loc[
                    excel_table["Unnamed: 0"].str.contains(
                        row["video_name"], case=False, na=False
                    )
                ]["Binned (Y/N)"].iloc[0]
            )
        else:
            binning.append("N")
    else:
        csv_file = glob(
            str(Path(row["tot_path_drop"]).parent)
            + f'/{row["Date Imaged"]}*{row["Plate number"]}.csv'
        )
        if len(csv_file) > 0:
            data_table.append(csv_file[0])
            df_comma = pd.read_csv(csv_file[0], nrows=1, sep=",")
            df_semi = pd.read_csv(csv_file[0], nrows=1, sep=";")
            if df_comma.shape[1] > df_semi.shape[1]:
                csv_table = pd.read_csv(csv_file[0], sep=",")
            else:
                csv_table = pd.read_csv(csv_file[0], sep=";")

            magnification.append(
                csv_table[csv_table["video"] == int(row["video_name"])]["Lens"].iloc[0]
            )
            fps.append(
                csv_table.loc[
                    csv_table["video"] == int(row["video_name"].split("_")[-1])
                ]["fps"].iloc[0]
            )
            mode.append(
                csv_table.loc[
                    csv_table["video"] == int(row["video_name"].split("_")[-1])
                ]["Illumination"].iloc[0]
            )
            binning.append(
                csv_table.loc[
                    csv_table["video"] == int(row["video_name"].split("_")[-1])
                ]["Binned"].iloc[0]
            )
        else:
            print("Halp! No datatable found! Pls add the excel file to the folders.")

dataframe["fps"] = fps
dataframe["magnification"] = magnification
dataframe["mode"] = mode
dataframe["parent_folder"] = [
    os.path.relpath(address, directory_targ) for address in dataframe["tot_path_drop"]
]
dataframe["data_table"] = data_table
dataframe["binned"] = binning

dataframe = dataframe.sort_values(by="video_name", ignore_index=True)
# dataframe = dataframe[dataframe['video_nr']== 9]
# dataframe = dataframe[dataframe['plate_nr'] == '046']
# print(len(dataframe))
# print(data_table)


if dataframe.isnull().values.any():
    print("Found NaNs in the excel files! Blame the experimentalists.")
    dataframe = dataframe.interpolate(method="pad", limit_direction="forward")
if dataframe.isnull().values.any():
    raise ("This excel sheet is unworkable, please ask the responsible person")

In [ ]:
excel_series = pd.Series(data_table).unique()
excel_series

In [ ]:
for xl_address in excel_series:
    file_name = "Analysis/" + os.path.relpath(xl_address, directory_targ)
    print(upload_targ + file_name)
    upload(xl_adress, upload_targ + file_name)